In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import json
import ast
import os
import csv
import math
from sklearn.utils import shuffle
from math import ceil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [6]:
tqdm.pandas()

In [7]:

args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "dataset_path":"/mnt/disks/data/fma/fma_large",
    "embeddings":"music_style",
    "sequence_size": 1280,
    "train_id": "hierarchical_all",
    'sample_size': 1
})


In [8]:

base_path = "/mnt/disks/data/fma/trains"


job_path = os.path.join(base_path,args.train_id)


tfrecord_path = os.path.join(job_path,"tfrecords")

# In[16]:

base_path = os.path.join(args.root_dir,"fma")

# In[17]:

models_path = os.path.join(args.root_dir,"models")


metadata_path_fma = os.path.join(base_path,"fma_metadata")

# In[18]:

metadata_path = os.path.join(job_path,"metadata.json")


categories_labels_path = os.path.join(job_path,"labels.json")


In [9]:
def __load_json__(path):
    with open(path, 'r') as f:
        tmp = json.loads(f.read())

    return tmp


## Load genres file. Contains relationships beetwen genres

In [13]:
genres_df = pd.read_csv(os.path.join(metadata_path_fma,'genres.csv'))


In [14]:
genres_df

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [15]:
genres_df[genres_df['genre_id'] == 495]

,genre_id,#tracks,parent,title,top_level
136,495,2061,15,Downtempo,15


## Análise do tracks.csv

In [16]:
tracks = os.path.join(metadata_path_fma,"tracks_valid.csv")

In [17]:
df_tracks = pd.read_csv(tracks)

In [18]:
df_tracks["track_genres_all"] = df_tracks.track_genres.apply(lambda x : ast.literal_eval(x))

In [19]:
genres_df['genre_id'] = genres_df.genre_id.astype(int)

In [20]:
genres_df['parent'] = genres_df.parent.astype(int)

## Join com tabela de generos

### Geração da hierarquia a partir das tracks


In [21]:
def get_structure(genres_id, df_genres):
    def get_from_df(genre_id, df_genres):
        if genre_id != 0:
            parent_genre = df_genres[df_genres["genre_id"]
                                     == genre_id].parent.values[0]
            return [genre_id, get_from_df(parent_genre, df_genres)]
    for genre_id in genres_id:
        print(get_from_df(genre_id, df_genres))

In [22]:
def get_structure(genres_id, df_genres):
    def get_from_df(genre_id, df_genres, output=[]):
        if genre_id != 0:
            parent_genre = df_genres[df_genres["genre_id"]
                                     == genre_id].parent.values[0]
            output.append(genre_id)
            get_from_df(parent_genre, df_genres, output=output)
            return output
    output_list = []
    for genre_id in genres_id:
        output_list.append(get_from_df(genre_id, df_genres, output=[]))
    return output_list

In [23]:
genres_df

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [24]:
get_structure([27,1032], genres_df)
get_structure([98,1032], genres_df)

[[98, 12], [1032, 102, 2]]

In [26]:
df_tracks.track_genres_all.iloc[256]

[27, 66]

In [27]:
df_tracks.iloc[158]

track_genres_all                                            [89]
track_genres                                                [89]
track_id_                                                    346
file_path           /mnt/disks/data/fma/fma_large/000/000346.mp3
Name: 158, dtype: object

In [ ]:
df_tracks["full_genre_id"] = df_tracks.track_genres_all.progress_apply(lambda x : get_structure(x,genres_df))

  0%|          | 0/106412 [00:00<?, ?it/s]

In [ ]:
df_tracks = df_tracks[['track_id_','full_genre_id']]

In [32]:
df_tracks.to_csv(os.path.join(metadata_path_fma,'tracks_valid.csv'),index=False)